In [40]:
from ortools.linear_solver import pywraplp
import numpy as np
from random import randint
import os
import glob

In [49]:
def transportationProblem(M,S,D,min=True):
    lenS= len(S)
    lenD= len(D)
    solver = pywraplp.Solver.CreateSolver('GLOP')
    #solver.use_dual_simplex = True
    V = [
        [solver.NumVar(0,solver.infinity(),'x{0}{1}'.format(i,j))
             for j in range(lenD)] 
                for i in range(lenS)
    ]
    Z = sum(M[i][j]*V[i][j] for i in range(lenS) for j in range(lenD))
    for i in range(lenS):
        solver.Add(
            sum(V[i][j] for j in range(lenD)) <= S[i]) 
    for j in range(lenD):
        solver.Add(
            sum(V[i][j] for i in range(lenS)) >= D[j])
    
    if(min):
        solver.Minimize(Z)
    else:
        solver.Maximize(Z)
    status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution:')
        print('Objective value =', solver.Objective().Value())
        print(['x{0}{1} = {2}'.format(i,j,round(V[i][j].solution_value())) for i in range(lenS) for j in range(lenD)])
        print('\nAdvanced usage:')
        print('Problem solved in %f milliseconds' % solver.wall_time())
        print('Problem solved in %d iterations' % solver.iterations())
    else:
        print('<The problem does not have an optimal solution.>')

In [48]:
def GenerateData(instances =10, lenS = 100, lenD=100, maxSend=20,  valid=False ):
    for i in range(instances):
        S = [randint(0,100) for i in range(lenS)]
        D = [randint(0,100) for i in range(lenD)]
        M = [ [randint(0,maxSend) for i in range(lenS)] for j in range(lenD)]
        print(S)
        print(D)
        print(M)
        transportationProblem(M,S,D)
        #files
        files = glob.glob('data/*')
        for f in files:
            os.remove(f)
        fo = open("data/dataS{0}.txt".format(i), "w")
        for x in range(lenS): fo.write('{0} '.format(S[x]))
        fo.close()
        fo = open("data/dataD{0}.txt".format(i), "w")
        for x in range(lenD): fo.write('{0} '.format(D[x]))
        fo.close()
        fo = open("data/data{0}.txt".format(i), "w")
        for x in range(lenS): 
            for y in range(lenD): 
                fo.write('{0} '.format(M[x][y]))
            fo.write('\n')
        fo.close()

GenerateData(1, 2, 2 )        

[20, 66]
[69, 75]
[[9, 8], [3, 10]]
<The problem does not have an optimal solution.>


In [9]:
M = np.loadtxt('data.txt')
S = np.loadtxt('dataS.txt')
D = np.loadtxt('dataD.txt')

transportationProblem(M,S,D)

Solution:
Objective value = 35.0
['x00 = 1', 'x01 = 6', 'x02 = 3', 'x10 = 0', 'x11 = 0', 'x12 = 0', 'x20 = 0', 'x21 = 0', 'x22 = 2', 'x30 = 8', 'x31 = 0', 'x32 = 0']


In [46]:
def transportationProblemDual(M,S,D):
    lenS = len(S)
    lenD = len(D)
    Z = [M[i][j] for i in range(lenS) for j in range(lenD)]
    #...


def dual(M):
    tamL = len(M)
    tamC = len(M[0])
    Z = M[0]
    M.pop(0)
    M_t = list(map(list, zip(*M))) # transposta de M
    D = M_t[tamL-1]
    M_t.pop(tamL-1)
    for i in range(len(M_t)):
        M_t[i].insert(tamC,Z[i])

    return M_t,D
    
    

dual([[1,2,3,0],
     [1,1,1,10],
     [2,1,4,12],
     [1,3,-1,9]])
    
    

transportationProblemDual([[0,5],
                           [2,6]],   [76,95],[69,81])
""" exemple

    [[0,5],
     [2,6]],   [76,95],[69,81]
     |
     v
    [[0,5,2,6,   0] # Z
     [1,1,0,0,  76],
     [0,0,1,1,  95],
     [1,0,1,0,  69],
     [0,1,0,1,  81]]
     

"""

' exemple\n\n    [[0,5],\n     [2,6]],   [76,95],[69,81]\n    [[1,1,0,0,  76],\n     [0,0,1,1,  95],\n     [1,0,1,0,  69],\n     [0,1,0,1,  81]],   [76,95],[69,81]\n     \n\n'

<generator object <genexpr> at 0x000001F7C88AED50>